In [34]:
from open3d import io
from open3d import geometry
from open3d import utility
from open3d import visualization
import meshcut
import scipy.interpolate as si
import splipy as sp
import numpy as np
import os
import math
import networkx as nx
import glob
import json
import ast
from geomdl import BSpline
from geomdl import multi
from geomdl import knotvector
from geomdl import operations
from ipywidgets import FloatProgress
from IPython.display import display

In [2]:
def create_pcd(points):
    pcd = geometry.PointCloud()
    pcd.points = utility.Vector3dVector(points)
    return pcd

In [5]:
folderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/NF_SCAN_V3/'
filedir = f'{folderdir}NF_REGULAR_V1.4.1_M2_ADJ.ply'
mesh = io.read_triangle_mesh(filedir)
vertices = np.asarray(mesh.vertices)
triangles = np.asarray(mesh.triangles)
mesh.compute_adjacency_list()
mesh.compute_vertex_normals()

In [6]:
folderdir = '/Users/nickf/Dropbox/Nicky T-Shirt Project/Subject Scans/Male Scans/Nicholas Ferrara/NF_LANDMARKS/'
landmarksdir = '/'.join(folderdir.split('/')[:-2])+'/'+'NF_LANDMARKS.json'
pomsdir = '/'.join(folderdir.split('/')[:-2])+'/'+'NF_POMS_V2.json'
with open(landmarksdir, 'r') as readlandmarks:
    landmarks = json.load(readlandmarks)
    with open(pomsdir, 'r') as readpoms:
        poms = json.load(readpoms)
        pom_landmarks = {}
        for key in poms:
            landmark_points = []
            for temp_key in poms[key]['landmarks']:
                landmark_points.append(landmarks[temp_key]['position'])
            landmark_dict = {
                'landmark_points': np.asarray(landmark_points),
                'periodic': poms[key]['periodic']
            }
            pom_landmarks[key] = landmark_dict

adj_list = mesh.adjacency_list
edges = []
for idx in range(len(vertices)):
    for n_idx in list(adj_list[idx]):
        edge = sorted([idx, n_idx])
        edges.append(edge)
edges = [(tuple(vertices[v1]), tuple(vertices[v2])) for v1, v2 in np.unique(np.asarray(edges), axis=0).tolist()]
G=nx.Graph()
G.add_edges_from(edges)

In [36]:
def path_weight(a, b):
    global point_a
    global point_b
    global normal
    u = np.array(b) - np.array(a)
    u_length = np.sqrt(np.dot(u,u))
    if type(normal) is type(None):
        v = np.array(point_b) - np.array(point_a) 
        v_length = np.sqrt(np.dot(v,v))
        tan = abs(math.tan(math.acos(abs(np.dot(u, v)) / (v_length*u_length+1e-6))))
        return tan*u_length*(tan**2 + u_length**2)**0.5
    else:
        n_length = np.sqrt(np.dot(normal,normal))
        tan = 1/abs(math.tan(math.acos(abs(np.dot(u, normal)) / (n_length*u_length+1e-6))))
        return tan*u_length*(tan**2 + u_length**2)**0.5

pom_paths = []
for key in pom_landmarks:
    points = pom_landmarks[key]['landmark_points']
    if key == 'TOTALNECK':
        normal = None
    else:
        normal = None
        #mean = points.mean(axis=0)
        #normal = np.linalg.svd(points - mean)[2][2]
    path = []
    if pom_landmarks[key]['periodic']:
        for i in range(-1, len(points)-1):
            point_a = points[i]
            point_b = points[i+1]
            path.extend(nx.astar_path(G, tuple(point_a), tuple(point_b), path_weight))
    else:
        for i in range(len(points)-1):
            point_a = points[i]
            point_b = points[i+1]
            path.extend(nx.astar_path(G, tuple(point_a), tuple(point_b), path_weight))
    pom_paths.append(np.array(path))

path_curves = []
for path in pom_paths:
    create_mesh = False
    if create_mesh:
        path_mesh = geometry.TriangleMesh()
        triangles_idx = []
        for vertex in path:
            idx = int(np.argwhere((vertices == vertex).all(axis=1)))
            for t_idx in np.argwhere((triangles == idx).any(1)).tolist():
                triangles_idx.extend(t_idx)
        path_mesh.vertices = utility.Vector3dVector(vertices)
        path_mesh.triangles = utility.Vector3iVector(np.unique(triangles[triangles_idx], axis=0))
    crv = BSpline.Curve()
    degree = 4
    if degree > len(path) - 1:
        degree = len(path) - 1
    crv.degree = degree
    crv.ctrlpts = path.tolist()
    crv.knotvector = knotvector.generate(crv.degree, crv.ctrlpts_size)
    crv.delta = 0.0005
    crv_points = crv.evalpts
    print(operations.length_curve(crv))
    path_curves.append(create_pcd(crv_points))
    if create_mesh:
        path_curves.append(path_mesh)

637.8681974632643
580.9969368061585
433.08479271681307
345.24893259381093
670.8767434042348
120.29614154571527
420.6769908727921
243.0317625101468
330.46286261974325
530.9262902346906
326.23742039127313
674.4453112416255
112.08248345876048
415.28643952503637
240.16908555477286
315.0843441256055
527.8494081523313
1011.6531432807348
371.93514929196897
382.7058662394
508.8235537518325
403.79343148096586
1081.7210166834702


In [13]:
visualization.draw_geometries(path_curves)

In [37]:
pom_landmarks.keys()

dict_keys(['CBLENGTH', 'CFLENGTH', 'CROSSSHOULDER', 'LBICEPBELOWAH', 'LFLENGTHFROMHPS', 'LSHOULDER', 'LSIDESEAM', 'LSLEEVE', 'LSLEEVEOPENING', 'LTOTALAH', 'RBICEPBELOWAH', 'RFLENGTHFROMHPS', 'RSHOULDER', 'RSIDESEAM', 'RSLEEVE', 'RSLEEVEOPENING', 'RTOTALAH', 'TOTALBELOWAH', 'FRONTBELOWHPS', 'BACKBELOWHPS', 'TOTALNECK', 'TOTALNECKBAND', 'TOTALSWEEP'])

In [33]:
from geomdl import visualization
help(visualization.VisMPL)

AttributeError: module 'geomdl.visualization' has no attribute 'VisMPL'